## kaggle 害虫数据集


In [1]:
from PIL import Image
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import time
from sklearn.model_selection import train_test_split
import shutil
from torchvision import datasets
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [3]:
#获取现在的路径
current_dir=os.getcwd()
print(current_dir)

/Users/weilinke/Desktop/github study/github_deep-learning/Pest_cnn


In [4]:
#数据所在的路径
data_dir=os.path.join(current_dir,'archive 2')
print(data_dir)

/Users/weilinke/Desktop/github study/github_deep-learning/Pest_cnn/archive 2


In [5]:
#做分割的，以及类别的获取
classes = os.listdir(data_dir)
print(classes)
print(len(classes))

['slug', 'snail', 'bees', 'ants', 'catterpillar', 'wasp', 'moth', 'grasshopper', 'weevil', 'beetle', 'earthworms', 'earwig']
12


In [6]:
#分割数据文件夹的位置
base_dir = os.path.join(data_dir,"splitted_data")
os.makedirs(base_dir, exist_ok=True)
print(base_dir)

/Users/weilinke/Desktop/github study/github_deep-learning/Pest_cnn/archive 2/splitted_data


In [7]:
#数据分割
for split in ['train', 'test','val']:
    for class_ in classes:
        os.makedirs(os.path.join(base_dir, split, class_), exist_ok=True)

In [8]:
#划分数据集
for cls in classes:
    #获取图片的地址，列出名字
    img_dir = os.path.join(data_dir, cls)
    images = os.listdir(img_dir)
    
    #训练验证测试集的划分
    train_imgs, val_test_imgs = train_test_split(images, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(val_test_imgs, test_size=0.33, random_state=42)
   
    #源文件拷贝到目标文件
    for img in train_imgs:
        shutil.copy(os.path.join(img_dir, img), os.path.join(base_dir, 'train', cls, img))
    
    for img in val_imgs:
        shutil.copy(os.path.join(img_dir, img), os.path.join(base_dir, 'val', cls, img))
        
    for img in test_imgs:
        shutil.copy(os.path.join(img_dir, img), os.path.join(base_dir, 'test', cls, img))

In [13]:
#拿一张图片看看数据类型
img = Image.open("/Users/weilinke/Desktop/github study/github_deep-learning/Pest_cnn/archive 2/splitted_data/train/ants/ants (1).jpg")

In [14]:
print(img,img.size)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x288 at 0x1064169B0> (300, 288)


In [11]:
#数据裁剪
transform=transforms.Compose([transforms.Resize((64,64)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [12]:
current_cid=os.getcwd()
print(current_cid)

/Users/weilinke/Desktop/github study/github_deep-learning/Pest_cnn


In [15]:
# 数据集的导入
train_dataset = datasets.ImageFolder(root='archive 2/splitted_data/train', transform=transform)
val_dataset = datasets.ImageFolder(root='archive 2/splitted_data/val', transform=transform)
test_dataset = datasets.ImageFolder(root='archive 2/splitted_data/test', transform=transform)

In [16]:
#拿一张图片看看数据类型
img =train_dataset[0][0]
print(img.shape)

torch.Size([3, 64, 64])


In [28]:
#定义模型
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.features=nn.Sequential(
            #第一层
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            #第二层

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            #第三层

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            #第四层

            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            #第五层
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.fc=nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*32*32,64),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
            nn.Linear(64,12)    
        )
    def forward(self,x):
        x=self.features(x)
        x=self.fc(x)
        return x


In [29]:
writer=SummaryWriter('./runs/pest_experiment_1')
input=torch.rand(64,3,64,64).float().to(device)
model=Model().to(device)
writer.add_graph(model,input)
writer.close()

In [22]:
# ! tensorboard --logdir=runs


In [23]:
#读取数据
train=DataLoader(train_dataset,batch_size=64,shuffle=True)
val=DataLoader(val_dataset,batch_size=64,shuffle=True)
test=DataLoader(test_dataset,batch_size=64,shuffle=True)

In [24]:
#加载优化器和定义损失函数
loss_fn=nn.CrossEntropyLoss()
loss_fn=loss_fn.to(device)
learning_rate=1e-3
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [30]:
#5.训练模型
writer=SummaryWriter("./pest_logs")
epochs=20

for epoch in range(epochs):
    print(f"-----第{epoch+1}轮训练开始了-----")
    
    start_time=time.time()
    #训练模型
    model.train()
    train_loss=0
    train_round=0

    for data in train:
        imgs ,labels= data 
        imgs,labels=imgs.to(device),labels.to(device)
        output=model(imgs)

        loss=loss_fn(output,labels)
        #定义优化器
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #记录损失
        train_loss+=loss.item()
        train_round+=1
        
        if train_round%10==0:
            print(f"第{train_round}迭代回合，损失为{train_loss/train_round:.4f}")
            writer.add_scalar("./train_loss",train_loss/train_round,global_step=train_round) 
    end_time=time.time()
    print(f"第{epoch+1}轮训练结束，耗时{end_time-start_time:.4f}秒")
    
    #测试模型
    model.eval()
    test_accuracy=0
    test_total_loss=0
    with torch.no_grad():
        for data in test:
            imgs,labels=data
            imgs,labels=imgs.to(device),labels.to(device)
            output=model(imgs)
            loss=loss_fn(output,labels)

            #记录正确率
            accuracy=(output.argmax(1)==labels).sum().item()
            test_total_loss+=loss.item()
            test_accuracy+=accuracy
    print(f"第{epoch+1}轮测试结束，正确率为{test_accuracy/len(test):.4f}")
    print(f"第{epoch+1}轮测试结束，损失为{test_total_loss:.4f}")
    writer.add_scalar("./test_loss",test_total_loss,global_step=epoch) 
    writer.add_scalar("./test_accuracy",test_accuracy/len(test),global_step=epoch)

    #保存模型
    save_floder="./pest_estimator"
    if not os.path.exists(save_floder):
        os.makedirs(save_floder)
    torch.save(model.state_dict(),os.path.join(save_floder,f"estimator_{epoch}.pth"))
    print("模型已保存")
    print(f"-----第{epoch+1}轮训练结束了-----")
    print()
writer.close()


-----第1轮训练开始了-----
第10迭代回合，损失为2.4849
第20迭代回合，损失为2.4841
第30迭代回合，损失为2.4855
第40迭代回合，损失为2.4851
第50迭代回合，损失为2.4867
第60迭代回合，损失为2.4866
第1轮训练结束，耗时12.4691秒
第1轮测试结束，正确率为4.8889
第1轮测试结束，损失为22.3796
模型已保存
-----第1轮训练结束了-----

-----第2轮训练开始了-----
第10迭代回合，损失为2.4857
第20迭代回合，损失为2.4859
第30迭代回合，损失为2.4866
第40迭代回合，损失为2.4861
第50迭代回合，损失为2.4858
第60迭代回合，损失为2.4866
第2轮训练结束，耗时11.9467秒
第2轮测试结束，正确率为4.8889
第2轮测试结束，损失为22.3794
模型已保存
-----第2轮训练结束了-----

-----第3轮训练开始了-----
第10迭代回合，损失为2.4823
第20迭代回合，损失为2.4833
第30迭代回合，损失为2.4844


KeyboardInterrupt: 

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.model = models.resnet18(pretrained=True)
#         self.model.fc = nn.Linear(512, 2)
        
#     def forward(self, x):
#         return self.model(x)